### 2018-03-23 Random Splitting by Tile and Size Filtering

Candidate plaques are grouped by their WSI-source, as well as their tile. This ensures that all plaques within a uniform 256 x 256 image are labeled (or multi-labeled) for the subsequent classification task. 

We use a pixel threshold of 1500 pixels, then randomly sample the tiled WSIs and copy them to a new directory for the dataset.

In [1]:
import os
import glob
import shutil

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import cv2

from tqdm import tqdm

In [2]:
BLOB_CSV = 'image_details.csv'
CSV_DIR = '/home/kangway/data/cnn_path/opencv_blobs/03_22_18/'

NORM_SRC =  '/home/kangway/data/cnn_path/opencv_blobs/03_22_18/blobs_bboxes/'
NORM_DEST = '/home/kangway/data/cnn_path/opencv_blobs/03_23_18_size_filtered/norm/blobs_bboxes/'

RAW_SRC   = '/home/kangway/data/cnn_path/opencv_blobs/03_23_18_raw/blobs/'
RAW_DEST  = '/home/kangway/data/cnn_path/opencv_blobs/03_23_18_size_filtered/raw/blobs/'

We will use a pixel threshold of 1500 pixels. We have a conversion rate of 0.5 microns per pixel, so 40 pixels wide = 20 microns wide. 

In [3]:
# Define a pixel threshold
THRESHOLD = 1500

In [4]:
# Load all the image data
image_df = pd.read_csv(CSV_DIR+BLOB_CSV)

In [5]:
image_df.head()

,imagename,source,tile_column,tile_row,image coordinates (xywh),blob coordinates (xywh),blob size
0,NA4619-02_AB_19_27_0.jpg,NA4619-02_AB,27,19,[653 0 256 256],[750 0 62 20],908
1,NA4619-02_AB_19_27_1.jpg,NA4619-02_AB,27,19,[794 39 256 256],[896 148 52 38],7088
2,NA4619-02_AB_19_27_2.jpg,NA4619-02_AB,27,19,[820 115 256 256],[920 224 56 38],1260
3,NA4619-02_AB_19_27_3.jpg,NA4619-02_AB,27,19,[994 163 256 256],[1108 276 28 30],1300
4,NA4619-02_AB_19_27_4.jpg,NA4619-02_AB,27,19,[1217 277 256 256],[1314 384 62 42],452


In [6]:
# Group by tile
grouped = image_df.groupby(['source', 'tile_column', 'tile_row'])

In [7]:
# This is a list of the group keys
tiles = list(grouped.groups)

In [8]:
# set a random seed
np.random.seed(42)

# shuffle the tiles
np.random.shuffle(tiles)

In [9]:
SPLIT = int(0.5 * len(tiles))

In [10]:
imageset = []
sources = []
for key in tiles[:SPLIT]:
    value = grouped.groups[key]
    images = image_df.loc[value]
    filtered = images[images['blob size'] > THRESHOLD]
    images = filtered['imagename']
    source = filtered['source']
    imageset.extend(images)
    sources.extend(source)

In [11]:
# down to 103409 images to label of greater than 1500 pixels
len(imageset)

103409

In [12]:
for image, source in tqdm(zip(imageset, sources)):
    rawdir = RAW_DEST + source
    normdir = NORM_DEST + source
    if not os.path.exists(rawdir):
        os.makedirs(rawdir)
    if not os.path.exists(normdir):
        os.makedirs(normdir)
    filename = source + "/" + image
    shutil.copy(RAW_SRC + filename, RAW_DEST + filename)
    shutil.copy(NORM_SRC + filename, NORM_DEST + filename)

103409it [01:24, 1222.50it/s]
